## Data transformation of EOS tables for MULTI-fs hydrodynamic code

[MULTI-fs](https://www.sciencedirect.com/science/article/pii/S0010465511003481?via%3Dihub) - hydrodynamic code for laser-plasma interaction in the femtosecond regime 
Structure of inverted equation of state (EOS) tables in MULTI-fs code:
1. Head line
  * Table ID 
  * Ignored (material density g/cm3)
  * NR - number of tabulated densities (g/cm3)
  * NE - number of tabulated internal energies (Mbar cm3/g)
2. Subsequent lines
  * NR values of the density (g/cm3)
  * NE values of internal energy variations (Mbar cm3/g)
  * NR values of e0 (cold internal energy at sufficiently small referene temperature) (Mbar cm3/g)
  * NR x NE values of pressure (Mbar)
  * NR x NE values of temperature (K)


In [1]:
import pandas as pd
import numpy as np

columns = ['Pressure','Concentration H2','Concentration H', 'Mass density rho', 'Entropy S', 'Internal Energy U',
           'log(drho)/log(T)','log(drho)/log(P)','log(dS)/log(T)', 'log(dS)/log(P)', 'log(dT)/log(P) adiabatic gradient']


df = pd.read_csv('h_tab_i.dat', sep='\s+', header=None, skiprows=1, names=columns)
header = pd.DataFrame([[2.10, 30]], columns = ['Pressure', 'Concentration H2'])
eos_table = header.append(df, ignore_index=True, sort=False)
#all logarithms are base10
print(eos_table.head())

   Pressure  Concentration H2  Concentration H  Mass density rho  Entropy S  \
0       2.1              30.0              NaN               NaN        NaN   
1       4.0               1.0              0.0           -5.7154     8.8910   
2       4.2               1.0              0.0           -5.5154     8.8803   
3       4.4               1.0              0.0           -5.3154     8.8693   
4       4.6               1.0              0.0           -5.1154     8.8580   

   Internal Energy U  log(drho)/log(T)  log(drho)/log(P)  log(dS)/log(T)  \
0                NaN               NaN               NaN             NaN   
1            10.1201              -1.0               1.0          0.1687   
2            10.1201              -1.0               1.0          0.1729   
3            10.1201              -1.0               1.0          0.1773   
4            10.1201              -1.0               1.0          0.1820   

   log(dS)/log(P)  log(dT)/log(P) adiabatic gradient  
0            

#### Electron inverted EOS table
**100100** - ID of the electron EOS table in MULTI fs input file

We will use first 93 values as a source for low temperature reference internal energy as NR and NE


In [2]:
#Header row
electron_table = pd.DataFrame([[100100, 0, 30, 93]], columns = ['A','B','C','D'])

#Subsequent lines

eos_table['Mass density rho'] = 10 ** eos_table['Mass density rho'] #Converting from log10 to g/cm3

reference_densities = eos_table['Mass density rho'][:32] #Selecting reference densities (3 null values)

print('Number of null values:', reference_densities.isnull().value_counts()[1])
reference_densities = reference_densities.dropna() 
reference_densities = reference_densities.reset_index(drop = True)
print(reference_densities)


Number of null values: 2
0     0.000002
1     0.000003
2     0.000005
3     0.000008
4     0.000012
5     0.000019
6     0.000031
7     0.000048
8     0.000077
9     0.000121
10    0.000192
11    0.000305
12    0.000483
13    0.000765
14    0.001212
15    0.001918
16    0.003032
17    0.004786
18    0.007530
19    0.011773
20    0.018168
21    0.027283
22    0.039030
23    0.052348
24    0.066145
25    0.080131
26    0.094167
27    0.108343
28    0.122999
29    0.138739
Name: Mass density rho, dtype: float64


In [3]:
# Add reference densities to the electron EOS table
i = 0
j = 1
test = {0:'A', 1:'B', 2:'C', 3:'D'}
for index, value in reference_densities.items():
    electron_table.loc[j,test[i]] = value
    i+=1
    if i == 4:
        i = 0
        j += 1
electron_table.fillna(0.)

,A,B,C,D
0,100100.000000,0.000000,30.000000,93.000000
1,0.000002,0.000003,0.000005,0.000008
2,0.000012,0.000019,0.000031,0.000048
3,0.000077,0.000121,0.000192,0.000305
4,0.000483,0.000765,0.001212,0.001918
5,0.003032,0.004786,0.007530,0.011773
6,0.018168,0.027283,0.039030,0.052348
7,0.066145,0.080131,0.094167,0.108343
8,0.122999,0.138739,0.000000,0.000000


In [4]:
# e0 values at small reference temperature
eos_table['Total energy'] = 10 ** eos_table['Internal Energy U'] #Converting from log10(ergs/g) to ergs/g
eos_table['Total energy'] = eos_table['Internal Energy U'] * (10 ** -4) #Converting ergs/g to m^2/s^2
eos_table['Total energy'] = eos_table['Internal Energy U'] / (10 ** 8) #Converting m^2/s^2 to Mbar*cm^3/g
reference_internal_energy_e0 = eos_table['Total energy'][:32] #Selecting reference densities (3 null values)

#print('Number of null values:', reference_internal_energy_e0.isnull().value_counts()[1])
#reference_internal_energy_e0 = reference_internal_energy_e0.dropna() 
#reference_internal_energy_e0 = reference_internal_energy_e0.reset_index(drop = True)
#print(reference_internal_energy_e0)